In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split    # Choose between models
from sklearn import preprocessing    # Scaling, transforming and wrangling data
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline    # Cross-validation
from sklearn.model_selection import GridSearchCV    # Cross-validation
from sklearn.metrics import mean_squared_error, r2_score    # Evaluation
#from sklearn.externals import joblib    # Persist the model. Alternative to pickle. Better for large numpy arrays

In [3]:
dataset_url = 'C:/Users/USER\Desktop/Decision/adult.csv'
data = pd.read_csv(dataset_url, sep=',')    # Load from remote

In [4]:
print (data.head)
print (data.shape)

<bound method NDFrame.head of        age         workclass   education  education-num      marital-status  \
0       39         State-gov   Bachelors             13       Never-married   
1       50  Self-emp-not-inc   Bachelors             13  Married-civ-spouse   
2       38           Private     HS-grad              9            Divorced   
3       53           Private        11th              7  Married-civ-spouse   
4       28           Private   Bachelors             13  Married-civ-spouse   
...    ...               ...         ...            ...                 ...   
32556   27           Private  Assoc-acdm             12  Married-civ-spouse   
32557   40           Private     HS-grad              9  Married-civ-spouse   
32558   58           Private     HS-grad              9             Widowed   
32559   22           Private     HS-grad              9       Never-married   
32560   52      Self-emp-inc     HS-grad              9  Married-civ-spouse   

       occupation   r

In [5]:
print(data.describe())
data.replace("?",np.nan,inplace=True)
data.dropna(inplace=True)
data['age'] = data['age'].astype(int)
data['hours-per-week'] = data['hours-per-week'].astype(int)
data.loc[data['class'] == '<=50K', 'class'] = 0
data.loc[data['class'] == '>50K', 'class'] = 1

                age  education-num  capital-gain  capital-loss  hours-per-week
count  32561.000000   32561.000000  32561.000000  32561.000000    32561.000000
mean      38.581647      10.080679   1077.648844     87.303830       40.437456
std       13.640433       2.572720   7385.292085    402.960219       12.347429
min       17.000000       1.000000      0.000000      0.000000        1.000000
25%       28.000000       9.000000      0.000000      0.000000       40.000000
50%       37.000000      10.000000      0.000000      0.000000       40.000000
75%       48.000000      12.000000      0.000000      0.000000       45.000000
max       90.000000      16.000000  99999.000000   4356.000000       99.000000


In [6]:
data

age         workclass   education  education-num      marital-status  \
0       39         State-gov   Bachelors             13       Never-married   
1       50  Self-emp-not-inc   Bachelors             13  Married-civ-spouse   
2       38           Private     HS-grad              9            Divorced   
3       53           Private        11th              7  Married-civ-spouse   
4       28           Private   Bachelors             13  Married-civ-spouse   
...    ...               ...         ...            ...                 ...   
32556   27           Private  Assoc-acdm             12  Married-civ-spouse   
32557   40           Private     HS-grad              9  Married-civ-spouse   
32558   58           Private     HS-grad              9             Widowed   
32559   22           Private     HS-grad              9       Never-married   
32560   52      Self-emp-inc     HS-grad              9  Married-civ-spouse   

       occupation   relationship   race     sex  capital-gain  capital-loss  \
0       Bachelors  Not-in-family  White    Male          2174             0   
1       Bachelors        Husband  White    Male             0             0   
2         HS-grad  Not-in-family  White    Male             0             0   
3            11th        Husband  Black    Male             0             0   
4       Bachelors           Wife  Black  Female             0             0   
...           ...            ...    ...     ...           ...           ...   
32556  Assoc-acdm           Wife  White  Female             0             0   
32557     HS-grad        Husband  White    Male             0             0   
32558     HS-grad      Unmarried  White  Female             0             0   
32559     HS-grad      Own-child  White    Male             0             0   
32560     HS-grad           Wife  White  Female         15024             0   

       hours-per-week native-country class  
0                  40  United-States     0  
1                  13  United-States     0  
2                  40  United-States     0  
3                  40  United-States     0  
4                  40           Cuba     0  
...               ...            ...   ...  
32556              38  United-States     0  
32557              40  United-States     1  
32558              40  United-States     0  
32559              20  United-States     0  
32560              40  United-States     1  

[30169 rows x 14 columns]

In [7]:
data_categorical= [i for i in data.columns
                       if data[i].dtype.name == 'object']
data_numerical = [i for i in data.columns
                     if data[i].dtype.name != 'object']

In [8]:
from sklearn.preprocessing import LabelEncoder 
le=LabelEncoder()

In [9]:
for col in data.columns:
    if data[col].dtypes == 'object':
        data[col] = le.fit_transform(data[col])

In [10]:
X=data.drop(['class'], axis=1)
y=data['class']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0,stratify=y)
X.shape

(30169, 13)

In [11]:
#標準化
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
X

age  workclass  education  education-num  marital-status  occupation  \
0       39          6          9             13               4           9   
1       50          5          9             13               2           9   
2       38          3         11              9               0          11   
3       53          3          1              7               2           1   
4       28          3          9             13               2           9   
...    ...        ...        ...            ...             ...         ...   
32556   27          3          7             12               2           7   
32557   40          3         11              9               2          11   
32558   58          3         11              9               6          11   
32559   22          3         11              9               4          11   
32560   52          4         11              9               2          11   

       relationship  race  sex  capital-gain  capital-loss  hours-per-week  \
0                 1     4    1          2174             0              40   
1                 0     4    1             0             0              13   
2                 1     4    1             0             0              40   
3                 0     2    1             0             0              40   
4                 5     2    0             0             0              40   
...             ...   ...  ...           ...           ...             ...   
32556             5     4    0             0             0              38   
32557             0     4    1             0             0              40   
32558             4     4    0             0             0              40   
32559             3     4    1             0             0              20   
32560             5     4    0         15024             0              40   

       native-country  
0                  38  
1                  38  
2                  38  
3                  38  
4                   4  
...               ...  
32556              38  
32557              38  
32558              38  
32559              38  
32560              38  

[30169 rows x 13 columns]

In [13]:
X_train

array([[ 1.10362956, -0.08685219,  0.17697488, ..., -0.21836787,
        -0.07642461,  0.26459636],
       [-0.94936622, -0.08685219, -0.34605074, ..., -0.21836787,
        -0.07642461,  0.26459636],
       [-1.17747686,  2.55206221,  1.22302613, ..., -0.21836787,
        -2.16165124,  0.26459636],
       ...,
       [-0.03692365,  1.67242408, -0.34605074, ..., -0.21836787,
         0.75766604,  0.26459636],
       [-1.17747686, -0.08685219,  1.22302613, ..., -0.21836787,
        -0.07642461,  0.26459636],
       [-1.10143998, -0.08685219, -1.39210199, ..., -0.21836787,
        -0.07642461,  0.26459636]])

In [14]:
scaler = preprocessing.StandardScaler().fit(X_train)  
# Save the mean and sd on the training dataset
#scaler = pd.concat([data[data_numerical],pd.get_dummies(data[data_categorical])], axis=1)

In [15]:
X_train_scaled = scaler.transform(X_train)
print (X_train_scaled.mean(axis=0))
print (X_train_scaled.std(axis=0))

[-3.65060283e-17 -3.10595644e-17  2.53186970e-17  7.02152237e-17
  1.00097174e-17  2.53186970e-17  4.71045526e-18 -5.00485871e-18
 -6.18247253e-17 -3.53284145e-18 -1.25121468e-16 -3.17955730e-17
 -8.24329671e-17]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [16]:
X_test_scaled = scaler.transform(X_test)
print (X_test_scaled.mean(axis=0))
print (X_test_scaled.std(axis=0))

[-0.01971013  0.00236533  0.01263007  0.02657644  0.01543776  0.01263007
  0.02830833  0.01350669  0.00966914 -0.02733741  0.00528389  0.00503538
 -0.00882698]
[0.99389522 1.01777406 0.98613504 0.99029974 0.99758621 0.98613504
 1.00679526 0.97764953 0.99633884 0.81875881 1.01962974 0.99694966
 1.02743042]


In [17]:
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=100))

In [18]:
print (pipeline.get_params())

{'memory': None, 'steps': [('standardscaler', StandardScaler()), ('randomforestregressor', RandomForestRegressor())], 'verbose': False, 'standardscaler': StandardScaler(), 'randomforestregressor': RandomForestRegressor(), 'standardscaler__copy': True, 'standardscaler__with_mean': True, 'standardscaler__with_std': True, 'randomforestregressor__bootstrap': True, 'randomforestregressor__ccp_alpha': 0.0, 'randomforestregressor__criterion': 'mse', 'randomforestregressor__max_depth': None, 'randomforestregressor__max_features': 'auto', 'randomforestregressor__max_leaf_nodes': None, 'randomforestregressor__max_samples': None, 'randomforestregressor__min_impurity_decrease': 0.0, 'randomforestregressor__min_impurity_split': None, 'randomforestregressor__min_samples_leaf': 1, 'randomforestregressor__min_samples_split': 2, 'randomforestregressor__min_weight_fraction_leaf': 0.0, 'randomforestregressor__n_estimators': 100, 'randomforestregressor__n_jobs': None, 'randomforestregressor__oob_score': F

In [19]:
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}

In [20]:
clf = GridSearchCV(pipeline, hyperparameters, cv=10)    # GridSearchCV: test out all possible permutations

In [21]:
# Fit and tune model
clf.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('randomforestregressor',
                                        RandomForestRegressor())]),
             param_grid={'randomforestregressor__max_depth': [None, 5, 3, 1],
                         'randomforestregressor__max_features': ['auto', 'sqrt',
                                                                 'log2']})

In [22]:
print (clf.best_params_)

{'randomforestregressor__max_depth': 5, 'randomforestregressor__max_features': 'auto'}


In [23]:
print (clf.refit)

True


In [24]:
y_pred = clf.predict(X_test)
y_pred

array([0.04165296, 0.87546722, 0.02996233, ..., 0.11321309, 0.67718138,
       0.03043678])

In [25]:
print (r2_score(y_test, y_pred))
print (mean_squared_error(y_test, y_pred))

0.4229750320773784
0.10788071786797895


In [54]:
from sklearn import metrics
print("MAPE",metrics.mean_squared_error(y_test, y_pred))
print("RMES",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


MAPE 0.10749323000999343
RMES 0.3278616019145783
